In [1]:
#!/usr/bin/env python

# Import modules from ase and fireworks
import os
import numpy as np

from ase.build import bulk
from ase.calculators.emt import EMT
from ase.db.core import connect

from fireworks import Firework, FWorker, PyTask, FWAction, Workflow

# Minimal hilde inputs to make dictionary conversion easier
from hilde.helpers.input_exchange import dict2patoms, patoms2dict
from hilde.helpers.hash import hash_atoms

# Get defaults from hilde.cfg
from hilde.settings import Settings

# Combined local/remote queue launching
from hilde.fireworks_api_adapter.combined_launcher import rapidfire as lq_rapidfire
from hilde.fireworks_api_adapter.launchpad import LaunchPadHilde as LaunchPad

# Import the hilde calculate function so both the local and remote machines have the same function in their path
from hilde.tasks.fireworks import calculate as hilde_calc

/home/purcell/bin/python/anaconda3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
mod_name = __name__

In [3]:
# Intialize Structures and database
ni = bulk("Ni", cubic=True)
ni.set_calculator(EMT())
ni_dict = patoms2dict(ni)

ni_hash, calc_hash = hash_atoms(ni)

db_path = os.getcwd() + "/test.db"

# Port changes are for my setup
launchpad = LaunchPad(port=27018, strm_lvl="INFO")

In [4]:
def calc_to_db(db_path, atoms_dict):
    db = connect(db_path)
    at = dict2patoms(atoms_dict[0])
    at.calc.atoms = at
    atoms_hash, calc_hash = hash_atoms(at)
    selection = [("atoms_hash", "=", atoms_hash), 
                 ("calc_hash", "=", calc_hash)]
    # Try to update the database if the material is already present, if not add it to the database
    try:
        rows = list(db.select(selection=selection))
        if not rows:
            raise KeyError()
        for row in rows:
            db.update(row.id, at, atoms_hash=atoms_hash, calc_hash=calc_hash)
    except KeyError:
        db.write(at, atoms_hash=atoms_hash, calc_hash=calc_hash)
    return FWAction()

In [5]:
q_spec = {
    # Submission script changes are controled by the _queueadapter dictionary
    "_queueadapter": {
        # Keys are the same that you define in "my_qadapter.yaml"
        "walltime": "00:01:00",
        "nodes": 1,
    }
}

In [22]:
# Remote Settings (Change these to match what you need)
remote_settings={
    # Has to be a full path to the host, draco.mpcdf.mpg.de will time out. Needs to be a list
    "remote_host": ["draco01.mpcdf.mpg.de"],
    # Not necessary if remote host is username@host
    "remote_user": "tpurcell",
    # Try to avoid using this
    "remote_password": None,
    "remote_config_dir": ["/u/tpurcell/.fireworks"],
    # If you are using Kerberos this needs to be True
    "gss_auth": True
}

In [37]:
# Remote Settings (Change these to match what you need)
settings = Settings("/home/purcell/git/hilde/hilde.cfg")
fireworks_kwargs = settings.fireworks_settings

In [38]:
# Set up a Workflow where each FireTask has its own Firework
wd = "/u/tpurcell/.fireworks/Ni/"

fw_calc = Firework(
    PyTask({"func": hilde_calc.name, "args": [wd, "calc_atoms", ni_dict]}), spec=q_spec
)

fw_to_db = Firework(
    PyTask(
        {"func": mod_name + ".calc_to_db", "args": [db_path], "inputs": ["calc_atoms"]}
    )
)
# Workflows defined by list of Fireworks, and a dict describing the links between each Firework
wf = Workflow([fw_calc, fw_to_db], {fw_calc: [fw_to_db]}, name="Example")

In [39]:
print(type(fireworks_kwargs["remote_host"]))

<class 'str'>


In [31]:
launchpad.add_wf(wf)
lq_rapidfire(
    launchpad,
    wflow=wf,
    launch_dir=".",
    nlaunches=0,
    njobs_queue=250,
    njobs_block=500,
    sleep_time=15,
    reserve=True,
    remote_host=fireworks_kwargs["remote_host"],
    remote_user=fireworks_kwargs["remote_user"],
    remote_password=fireworks_kwargs["remote_password"],
    remote_config_dir=fireworks_kwargs["remote_config_dir"],
    # List of tasks that should be done on the queue
    tasks2queue=[hilde_calc.name],
    gss_auth=fireworks_kwargs["gss_auth"],
)

2018-11-16 15:27:29,053 INFO Added a workflow. id_map: {75: 77, 76: 78}
78 [78, 77]
qlaunch_hilde --reserve rapidfire --maxjobs_queue 250 --maxjobs_block 500 --nlaunches 1 --sleep 15 --firework_ids 78
2018-11-16 15:27:29,319 ERROR ----|vvv|----
2018-11-16 15:27:29,320 ERROR Error with queue launcher rapid fire!
2018-11-16 15:27:29,323 ERROR Traceback (most recent call last):
  File "/home/purcell/bin/python/anaconda3.6/lib/python3.6/site-packages/HiLDe-0.1-py3.6-linux-x86_64.egg/hilde/fireworks_api_adapter/combined_launcher.py", line 237, in rapidfire
    return_code = rlaunch(*args, **kwargs)
  File "/home/purcell/bin/python/anaconda3.6/lib/python3.6/site-packages/HiLDe-0.1-py3.6-linux-x86_64.egg/hilde/fireworks_api_adapter/qlaunch_remote.py", line 143, in qlaunch_remote
    "qlaunch_hilde {} {} {}".format(pre_non_default, command, non_default)
  File "<decorator-gen-3>", line 2, in run
  File "/home/purcell/bin/python/anaconda3.6/lib/python3.6/site-packages/fabric/connection.py", lin

In [9]:
# See the results with the database access
db = connect(db_path)
at = db.get_atoms(calc_hash=calc_hash, atoms_hash=ni_hash, attach_calculator=True)
row = list(db.select(selection=[("atoms_hash", "=", ni_hash)], columns=["forces"]))[0]
print(f"Atomic forces from the new atoms are: \n{at.get_forces()}")
print(f"Atomic forces from the row is: \n{row.forces}")

Atomic forces from the new atoms are: 
[[ 2.75670600e-15 -1.06217932e-16 -5.86336535e-16]
 [ 2.08575155e-14 -2.32452946e-15 -1.40512602e-15]
 [-2.75474088e-15  2.34534614e-15  1.25940924e-15]
 [-9.96193416e-15 -2.80117861e-15  2.01227923e-16]]
Atomic forces from the row is: 
[[ 2.75670600e-15 -1.06217932e-16 -5.86336535e-16]
 [ 2.08575155e-14 -2.32452946e-15 -1.40512602e-15]
 [-2.75474088e-15  2.34534614e-15  1.25940924e-15]
 [-9.96193416e-15 -2.80117861e-15  2.01227923e-16]]
